In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    i=1
    for filename in filenames:
        print(os.path.join(dirname, filename))
        i+=1
        if i==20:
            break

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
tf.__version__

In [ ]:
# detect and init the TPU
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()

# instantiate a distribution strategy
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.TPUStrategy(tpu)

In [ ]:
summary_df = pd.read_csv('/kaggle/input/coronahack-chest-xraydataset/Chest_xray_Corona_dataset_Summary.csv')
metadata_df = pd.read_csv('/kaggle/input/coronahack-chest-xraydataset/Chest_xray_Corona_Metadata.csv')

# Define the image directory
path = '/kaggle/input/coronahack-chest-xraydataset/Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/'


image_size = 224
batch_size = 128
channel_size = 3 #RGB

In [ ]:
metadata_df.head()

In [ ]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1/255.0,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # Adjust the split ratio as needed
)

In [ ]:
metadata_df[metadata_df['Dataset_type'] == 'TRAIN']

In [ ]:
metadata_df["all_labels"] = metadata_df[['Label', 'Label_1_Virus_category', 'Label_2_Virus_category']].apply(lambda x: ','.join(x.dropna().astype(str)), axis=1)

In [ ]:
metadata_df.head()

In [ ]:
metadata_df['all_labels'].value_counts()

In [ ]:
# Train data generator
train_generator = datagen.flow_from_dataframe(
    dataframe=metadata_df[metadata_df['Dataset_type'] == 'TRAIN'],
    directory=path+"/train/",
    x_col="X_ray_image_name",
    y_col="all_labels",
    target_size=(image_size,image_size),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)


In [ ]:
# Validation data generator
validation_generator = datagen.flow_from_dataframe(
    dataframe=metadata_df[metadata_df['Dataset_type'] == 'TRAIN'],
    directory=path+"/train/",
    x_col="X_ray_image_name",
    y_col="all_labels",
    target_size=(image_size,image_size),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)


In [ ]:
# Test data generator
test_generator = datagen.flow_from_dataframe(
    dataframe=metadata_df[metadata_df['Dataset_type'] == 'TEST'],
    directory=path+"/test/",
    x_col="X_ray_image_name",
    y_col="all_labels",
    target_size=(image_size,image_size),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False  # No need to shuffle test data
)